In [1]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import datetime
from datetime import timedelta

In [2]:
!sudo apt-get install python3-dev default-libmysqlclient-dev
!pip install -q pymysql

Reading package lists... Done
Building dependency tree       
Reading state information... Done
default-libmysqlclient-dev is already the newest version (1.0.4).
default-libmysqlclient-dev set to manually installed.
python3-dev is already the newest version (3.6.7-1~18.04).
python3-dev set to manually installed.
The following package was automatically installed and is no longer required:
  libnvidia-common-470
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.
     |████████████████████████████████| 43 kB 1.7 MB/s 


In [3]:
from google.colab import drive
drive.mount('/content/drive')

PATH = '/content/drive/My Drive/Colab Notebooks/SOYO/Election/'

Mounted at /content/drive


In [4]:
import pymysql

host = 'database-president.cvvsira8ejfm.ap-northeast-2.rds.amazonaws.com'
port = 3306
username = 'jun'
password = 'dragoneye'
db = 'president'

conn = pymysql.connect(host=host, port=port, user=username, password=password, db=db)

In [5]:
sql_state = 'SELECT * FROM search_trend'

search_df = pd.read_sql_query(sql_state, conn)
conn.close()

In [6]:
# naver datalab
naver_df = search_df.loc[search_df['source_id'] == 1]

# google trends
google_df = search_df.loc[search_df['source_id'] == 2]

# total만 추출
naver_df = naver_df.loc[naver_df['search_condition'] == 'total']
google_df = google_df.loc[google_df['search_condition'] == 'total']
naver_df.reset_index(drop=True, inplace=True)
google_df.reset_index(drop=True, inplace=True)

In [8]:
naver_df.tail()

,id,date,val_1,val_2,search_condition,election_id,source_id
571687,20345373,2022-02-27,60.8696,15.94200,total,764,1
571688,20345374,2022-02-28,100.0000,14.49280,total,764,1
571689,20345375,2022-03-01,56.5217,7.24637,total,764,1
571690,20345376,2022-03-02,69.5652,7.24637,total,764,1
571691,20345377,2022-03-03,85.5072,8.69565,total,764,1


In [9]:
google_df.tail()

,id,date,val_1,val_2,search_condition,election_id,source_id
74031,20312766,2022-02-27,0.0,0.0,total,764,2
74032,20312767,2022-02-28,0.0,0.0,total,764,2
74033,20312768,2022-03-01,0.0,0.0,total,764,2
74034,20312769,2022-03-02,0.0,0.0,total,764,2
74035,20312770,2022-03-03,0.0,0.0,total,764,2


In [7]:
# 578은 네이버 데이터렙에만 있고 구글 트랜드에는 없는 후보쌍
set(naver_df['election_id']).difference(set(google_df['election_id']))

set()

In [20]:
naver_data = naver_df.loc[naver_df['election_id'] != 578] # 578번 후보쌍 제거
naver_data.reset_index(drop=True, inplace=True)

# 7일 단위로 찍힌 구글 데이터
google_data1 = google_df.loc[google_df['date'] < datetime.datetime(2022, 1, 23)]
google_data1.reset_index(drop=True, inplace=True)

# 1일 단위로 찍힌 구글 데이터
google_data2 = google_df.loc[google_df['date'] >= datetime.datetime(2022, 1, 23)]
google_data2.reset_index(drop=True, inplace=True)

In [10]:
# 2/18 ~ 3/3(2주간)
naver_data = naver_df.loc[(naver_df['date'] >= datetime.datetime(2022, 2, 18)) & (naver_df['date'] <= datetime.datetime(2022, 3, 3))]
google_data = google_df.loc[(google_df['date'] >= datetime.datetime(2022, 2, 18)) & (google_df['date'] <= datetime.datetime(2022, 3, 3))]
print(naver_data.shape)
print(google_data.shape)

(3226, 7)
(3228, 7)


In [17]:
# 후보쌍 10개가 차이남
set(google_data['election_id']).difference(set(naver_data['election_id']))

{583, 587, 636, 645, 682, 689, 706, 744, 749, 755}

# 네이버 전처리
* 3/3까지 추가된 데이터를 기준으로 다시 전처리 실행

In [21]:
naver_result = pd.DataFrame()
flag = 0
for id in naver_data['election_id'].unique():
  print(f'{flag + 1}번째 실행 시작')
  tmp_result = pd.DataFrame()
  date_id = datetime.datetime(2016, 1, 1) # 초기 날짜
  can_df = naver_data.loc[naver_data['election_id'] == id]

  # 변수 초기화
  init = can_df.loc[(can_df['date'] < date_id + timedelta(7)) & (can_df['date'] >= date_id)]

  total0 = np.sum(init.loc[init['search_condition'] == 'total', 'val_1'].values)
  total1 = np.sum(init.loc[init['search_condition'] == 'total', 'val_2'].values)

  date_id = date_id + timedelta(7) # loop를 위한 날짜 초기화(일주일 뒤)
  while True:
    # 일주일 단위로 묶음
    tmp_df = can_df.loc[(can_df['date'] < date_id + timedelta(7)) & (can_df['date'] >= date_id)]

    total0 = np.sum(tmp_df.loc[tmp_df['search_condition'] == 'total', 'val_1'].values) - total0 # 다음 일주일 데이터 - 현재 일주일 데이터
    total1 = np.sum(tmp_df.loc[tmp_df['search_condition'] == 'total', 'val_2'].values) - total1

    # 후보1 - 후보2
    total = total0 - total1

    index_list = [id]
    row_list = [total]
    column_list = [datetime.datetime.strftime(date_id, format='%Y-%m-%d') + '_' + datetime.datetime.strftime(date_id + timedelta(6), format='%Y-%m-%d')]
    temp = pd.DataFrame(row_list, index=index_list, columns=column_list)
    tmp_result = pd.concat([tmp_result, temp], axis=1)

    # update total value for next loop
    total0 = np.sum(tmp_df.loc[tmp_df['search_condition'] == 'total', 'val_1'].values)
    total1 = np.sum(tmp_df.loc[tmp_df['search_condition'] == 'total', 'val_2'].values)

    date_id = date_id + timedelta(7) # update date
     # while문 종료 조건: search_df 날짜의 최댓값보다 클 경우
    if date_id > datetime.datetime.fromisoformat(str(max(naver_data['date'].unique()))[:10]):
      break

  print(f'{flag + 1}번째 실행 완료')
  naver_result = pd.concat([naver_result, tmp_result])
  flag += 1

1번째 실행 시작
1번째 실행 완료
2번째 실행 시작
2번째 실행 완료
3번째 실행 시작
3번째 실행 완료
4번째 실행 시작
4번째 실행 완료
5번째 실행 시작
5번째 실행 완료
6번째 실행 시작
6번째 실행 완료
7번째 실행 시작
7번째 실행 완료
8번째 실행 시작
8번째 실행 완료
9번째 실행 시작
9번째 실행 완료
10번째 실행 시작
10번째 실행 완료
11번째 실행 시작
11번째 실행 완료
12번째 실행 시작
12번째 실행 완료
13번째 실행 시작
13번째 실행 완료
14번째 실행 시작
14번째 실행 완료
15번째 실행 시작
15번째 실행 완료
16번째 실행 시작
16번째 실행 완료
17번째 실행 시작
17번째 실행 완료
18번째 실행 시작
18번째 실행 완료
19번째 실행 시작
19번째 실행 완료
20번째 실행 시작
20번째 실행 완료
21번째 실행 시작
21번째 실행 완료
22번째 실행 시작
22번째 실행 완료
23번째 실행 시작
23번째 실행 완료
24번째 실행 시작
24번째 실행 완료
25번째 실행 시작
25번째 실행 완료
26번째 실행 시작
26번째 실행 완료
27번째 실행 시작
27번째 실행 완료
28번째 실행 시작
28번째 실행 완료
29번째 실행 시작
29번째 실행 완료
30번째 실행 시작
30번째 실행 완료
31번째 실행 시작
31번째 실행 완료
32번째 실행 시작
32번째 실행 완료
33번째 실행 시작
33번째 실행 완료
34번째 실행 시작
34번째 실행 완료
35번째 실행 시작
35번째 실행 완료
36번째 실행 시작
36번째 실행 완료
37번째 실행 시작
37번째 실행 완료
38번째 실행 시작
38번째 실행 완료
39번째 실행 시작
39번째 실행 완료
40번째 실행 시작
40번째 실행 완료
41번째 실행 시작
41번째 실행 완료
42번째 실행 시작
42번째 실행 완료
43번째 실행 시작
43번째 실행 완료
44번째 실행 시작
44번째 실행 완료
45번째 실행 시작
45번째 실행 완료
46번째 실행 시작
46번째 실행 완료
47번째 실

In [22]:
naver_result

,2016-01-08_2016-01-14,2016-01-15_2016-01-21,2016-01-22_2016-01-28,2016-01-29_2016-02-04,2016-02-05_2016-02-11,2016-02-12_2016-02-18,2016-02-19_2016-02-25,2016-02-26_2016-03-03,2016-03-04_2016-03-10,2016-03-11_2016-03-17,...,2021-12-24_2021-12-30,2021-12-31_2022-01-06,2022-01-07_2022-01-13,2022-01-14_2022-01-20,2022-01-21_2022-01-27,2022-01-28_2022-02-03,2022-02-04_2022-02-10,2022-02-11_2022-02-17,2022-02-18_2022-02-24,2022-02-25_2022-03-03
638,1.04666,0.84136,-3.29526,0.59096,0.07511,-0.32052,0.29548,3.74098,1.83795,11.88898,...,-13.58677,-1.37723,-0.04506,-0.88640,155.87625,-77.37519,14.14138,128.43661,-82.47818,74.13402
532,0.14023,0.05686,-0.45859,0.26529,-0.37520,0.55333,-0.34867,0.50785,1.78123,100.19333,...,-1.26585,0.73524,7.88677,-8.19373,78.64891,3.02975,157.32182,100.95593,106.41564,-40.07484
744,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
539,-0.01944,0.07481,-0.14800,-0.00592,0.02484,-0.09049,0.00189,-0.03458,-0.01862,-0.40352,...,-1.08525,-0.30249,0.34731,-0.25953,94.82576,316.35931,-157.65770,-162.98420,-166.29674,417.02828
576,-0.00078,-0.00258,0.01395,-0.00955,-0.00284,0.00257,0.00387,-0.00800,0.00206,-0.00025,...,0.20226,-0.02531,-0.01911,0.04364,-253.42217,68.90839,88.62871,71.57196,-42.89305,-214.42314
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
577,-0.05681,-0.01071,0.01358,-0.00212,0.04149,-0.01801,-0.00914,9.53001,-9.48141,-0.14483,...,0.51955,-0.22853,0.49527,0.11790,-200.78705,146.74380,18.81385,27.96407,2.87456,-260.11227
657,-0.21421,0.23564,-0.44833,-0.86604,0.72225,0.02105,-0.29625,0.68796,-0.00819,-0.33141,...,0.00482,-0.02126,0.11011,-0.15212,-49.88488,-112.92539,140.23880,-172.65680,35.80440,46.57490
667,0.48364,0.45382,-5.69761,-3.57427,10.77910,0.55983,-0.05300,0.33458,0.78839,0.20867,...,0.92750,-2.88526,-0.78177,3.26951,1.16273,0.75858,0.00000,-97.16786,-55.73988,0.98472
720,0.14474,-0.05941,-0.21118,-2.21863,2.05362,0.16432,-0.52512,0.74676,0.37689,0.77125,...,-0.15524,0.18740,-1.54598,2.40460,-56.74789,-90.70097,-35.33661,-83.16010,-57.19030,84.85175


In [23]:
naver_result.to_csv(PATH + 'dataset/naver_total.csv', index=True)

# 구글 전처리

## 7일 단위로 적재된 데이터

In [ ]:
google_result1 = pd.DataFrame()
flag = 0
for id in google_data1['election_id'].unique():
  print(f'{flag + 1}번째 실행 시작')
  tmp_result = pd.DataFrame()
  can_df = google_data1.loc[google_data1['election_id'] == id]

  total0_values = can_df['val_1'].values
  total1_values = can_df['val_2'].values

  date_values = can_df['date']

  total0 = total0_values[0]
  total1 = total1_values[0]

  i = 0
  while True:
    i += 1
    total0 = total0_values[i] - total0 # 다음 일주일 데이터 - 현재 일주일 데이터
    total1 = total1_values[i] - total1

    total = total0 - total1

    index_list = [id]
    row_list = [total]
    column_list = [datetime.datetime.strftime(date_values.iloc[i - 1], format='%Y-%m-%d') + '_' + datetime.datetime.strftime(date_values.iloc[i - 1] + timedelta(6), format='%Y-%m-%d')]
    temp = pd.DataFrame(row_list, index=index_list, columns=column_list)
    tmp_result = pd.concat([tmp_result, temp], axis=1)

    # update total value for next loop
    total0 = total0_values[i]
    total1 = total1_values[i]

     # while문 종료 조건: values 값 끝까지 간 경우
    if i == len(total0_values) - 1:
      break

  print(f'{flag + 1}번째 실행 완료')
  google_result1 = pd.concat([google_result1, tmp_result])
  flag += 1

1번째 실행 시작
1번째 실행 완료
2번째 실행 시작
2번째 실행 완료
3번째 실행 시작
3번째 실행 완료
4번째 실행 시작
4번째 실행 완료
5번째 실행 시작
5번째 실행 완료
6번째 실행 시작
6번째 실행 완료
7번째 실행 시작
7번째 실행 완료
8번째 실행 시작
8번째 실행 완료
9번째 실행 시작
9번째 실행 완료
10번째 실행 시작
10번째 실행 완료
11번째 실행 시작
11번째 실행 완료
12번째 실행 시작
12번째 실행 완료
13번째 실행 시작
13번째 실행 완료
14번째 실행 시작
14번째 실행 완료
15번째 실행 시작
15번째 실행 완료
16번째 실행 시작
16번째 실행 완료
17번째 실행 시작
17번째 실행 완료
18번째 실행 시작
18번째 실행 완료
19번째 실행 시작
19번째 실행 완료
20번째 실행 시작
20번째 실행 완료
21번째 실행 시작
21번째 실행 완료
22번째 실행 시작
22번째 실행 완료
23번째 실행 시작
23번째 실행 완료
24번째 실행 시작
24번째 실행 완료
25번째 실행 시작
25번째 실행 완료
26번째 실행 시작
26번째 실행 완료
27번째 실행 시작
27번째 실행 완료
28번째 실행 시작
28번째 실행 완료
29번째 실행 시작
29번째 실행 완료
30번째 실행 시작
30번째 실행 완료
31번째 실행 시작
31번째 실행 완료
32번째 실행 시작
32번째 실행 완료
33번째 실행 시작
33번째 실행 완료
34번째 실행 시작
34번째 실행 완료
35번째 실행 시작
35번째 실행 완료
36번째 실행 시작
36번째 실행 완료
37번째 실행 시작
37번째 실행 완료
38번째 실행 시작
38번째 실행 완료
39번째 실행 시작
39번째 실행 완료
40번째 실행 시작
40번째 실행 완료
41번째 실행 시작
41번째 실행 완료
42번째 실행 시작
42번째 실행 완료
43번째 실행 시작
43번째 실행 완료
44번째 실행 시작
44번째 실행 완료
45번째 실행 시작
45번째 실행 완료
46번째 실행 시작
46번째 실행 완료
47번째 실

## 1일 단위로 적재된 데이터

In [24]:
google_result2 = pd.DataFrame()
flag = 0
for id in google_data2['election_id'].unique():
  print(f'{flag + 1}번째 실행 시작')
  tmp_result = pd.DataFrame()
  date_id = datetime.datetime(2022, 1, 16) # 초기 날짜
  can_df = google_data2.loc[google_data2['election_id'] == id]

  # 변수 초기화
  init = can_df.loc[(can_df['date'] < date_id + timedelta(7)) & (can_df['date'] >= date_id)]

  total0 = np.sum(init.loc[init['search_condition'] == 'total', 'val_1'].values)
  total1 = np.sum(init.loc[init['search_condition'] == 'total', 'val_2'].values)

  date_id = date_id + timedelta(7) # loop를 위한 날짜 초기화(일주일 뒤)
  while True:
    # 일주일 단위로 묶음
    tmp_df = can_df.loc[(can_df['date'] < date_id + timedelta(7)) & (can_df['date'] >= date_id)]

    total0 = np.sum(tmp_df.loc[tmp_df['search_condition'] == 'total', 'val_1'].values) - total0 # 다음 일주일 데이터 - 현재 일주일 데이터
    total1 = np.sum(tmp_df.loc[tmp_df['search_condition'] == 'total', 'val_2'].values) - total1

    # 후보1 - 후보2
    total = total0 - total1
    
    index_list = [id]
    row_list = [total]
    column_list = [datetime.datetime.strftime(date_id, format='%Y-%m-%d') + '_' + datetime.datetime.strftime(date_id + timedelta(6), format='%Y-%m-%d')]
    temp = pd.DataFrame(row_list, index=index_list, columns=column_list)
    tmp_result = pd.concat([tmp_result, temp], axis=1)

    # update total value for next loop
    total0 = np.sum(tmp_df.loc[tmp_df['search_condition'] == 'total', 'val_1'].values)
    total1 = np.sum(tmp_df.loc[tmp_df['search_condition'] == 'total', 'val_2'].values)

    date_id = date_id + timedelta(7) # update date
     # while문 종료 조건: search_df 날짜의 최댓값보다 클 경우
    if date_id > datetime.datetime.fromisoformat(str(max(google_data2['date'].unique()))[:10]):
      break

  print(f'{flag + 1}번째 실행 완료')
  google_result2 = pd.concat([google_result2, tmp_result])
  flag += 1

1번째 실행 시작
1번째 실행 완료
2번째 실행 시작
2번째 실행 완료
3번째 실행 시작
3번째 실행 완료
4번째 실행 시작
4번째 실행 완료
5번째 실행 시작
5번째 실행 완료
6번째 실행 시작
6번째 실행 완료
7번째 실행 시작
7번째 실행 완료
8번째 실행 시작
8번째 실행 완료
9번째 실행 시작
9번째 실행 완료
10번째 실행 시작
10번째 실행 완료
11번째 실행 시작
11번째 실행 완료
12번째 실행 시작
12번째 실행 완료
13번째 실행 시작
13번째 실행 완료
14번째 실행 시작
14번째 실행 완료
15번째 실행 시작
15번째 실행 완료
16번째 실행 시작
16번째 실행 완료
17번째 실행 시작
17번째 실행 완료
18번째 실행 시작
18번째 실행 완료
19번째 실행 시작
19번째 실행 완료
20번째 실행 시작
20번째 실행 완료
21번째 실행 시작
21번째 실행 완료
22번째 실행 시작
22번째 실행 완료
23번째 실행 시작
23번째 실행 완료
24번째 실행 시작
24번째 실행 완료
25번째 실행 시작
25번째 실행 완료
26번째 실행 시작
26번째 실행 완료
27번째 실행 시작
27번째 실행 완료
28번째 실행 시작
28번째 실행 완료
29번째 실행 시작
29번째 실행 완료
30번째 실행 시작
30번째 실행 완료
31번째 실행 시작
31번째 실행 완료
32번째 실행 시작
32번째 실행 완료
33번째 실행 시작
33번째 실행 완료
34번째 실행 시작
34번째 실행 완료
35번째 실행 시작
35번째 실행 완료
36번째 실행 시작
36번째 실행 완료
37번째 실행 시작
37번째 실행 완료
38번째 실행 시작
38번째 실행 완료
39번째 실행 시작
39번째 실행 완료
40번째 실행 시작
40번째 실행 완료
41번째 실행 시작
41번째 실행 완료
42번째 실행 시작
42번째 실행 완료
43번째 실행 시작
43번째 실행 완료
44번째 실행 시작
44번째 실행 완료
45번째 실행 시작
45번째 실행 완료
46번째 실행 시작
46번째 실행 완료
47번째 실

In [25]:
# google_result1.to_csv(PATH + 'dataset/google_total_0127.csv', index=True)
google_result2.to_csv(PATH + 'dataset/google_total_0303.csv', index=True)